In [1]:
import pandas as pd

from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
rs = 7812

report_date - дата среза фичей

есть есть еденица мы не знаем какая она 1 и 2 или 3е

In [3]:
raw_data = pd.read_csv('data/train_dataset_Самолет.csv', low_memory=False)

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14456 entries, 0 to 14455
Columns: 2666 entries, report_date to col2663
dtypes: float64(2312), int64(3), object(351)
memory usage: 294.0+ MB


In [5]:
raw_data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,2022-07-01,1241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,2022-09-01,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,2022-02-01,7116,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,2021-08-01,7117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


In [6]:
raw_data['target'].value_counts()

0    14020
1      436
Name: target, dtype: int64

In [7]:
num_unq_clients = raw_data['client_id'].unique().shape[0] # Number of unque clients
num_unq_clients

4817

In [8]:
raw_data.iloc[1,3:-1].dropna().unique()

array([1.0, 0.0, 4.0, 19.2, 19.200000762939453, 92.55, 92.5500030517578,
       30.0, 5407132.0, 23789540.0, 715.0,
       'https://samolet.ru/flats/?project=7', 'filter_price_asc', 55.7483,
       55.74829864501953, 37.6171, 37.61709976196289, 'page_load',
       'Mobile Safari', 'iPhone', 'iOS', '/project/sputnik/', 'sms',
       'exponea', 'Russia', 'Moscow',
       '01febac0-b083-494e-8589-f98400074b94', 6.0, 2.0, 8193703.78,
       5000.0, 1420568.84333333, 277709.27, 92569.75666667, 3.0, 3.5,
       204857.0, 95379.0, 150118.0, 7938208.75, 318565.86, 4128387.305,
       38.75, 3.34, 21.045, 242.0, 5903704.0, 362.0, 7616803.0],
      dtype=object)

In [9]:
data = raw_data.drop(columns=['report_date','client_id'])
data

,target,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


-----------

Удаление столбцов с хеш-суммами, uuid

In [10]:
# Regexp для поиска и удаления хеш сумм, так как это уникальное значение
pattern = r'^[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{12}$'

# Функция для фильтрации колонок по регулярному выражению
def filter_columns(column_name):
    return not data[column_name].astype(str).str.match(pattern).any()

raw_data_wout_regex = data[data.columns[data.columns.to_series().apply(filter_columns)]]

In [11]:
datat = raw_data_wout_regex.copy()
datat

,target,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


--------------

Split on train test raw data

In [12]:
datat.drop(columns=['target'])

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


In [13]:
X_train, X_test, y_train, y_test = train_test_split(datat.drop(columns=['target']), datat['target'], test_size=0.3, random_state=rs+20)

In [14]:
Train_data = X_train
Train_data['target'] = y_train
Test_data = X_test
Test_data['target'] = y_test


--------------

Удаление строк с целевым признаком 0, у которых большинство (медианное значение) признаков Nan

In [15]:
# Надо посчитать сколько в среднем заполнено значений для цп 0
median_t0 = Train_data[Train_data['target'] == 0].iloc[:,1:].apply(lambda row: row.notna().sum(), axis=1).median()
median_t0

355.0

In [16]:
data = Train_data[~((Train_data['target'] == 0) & (Train_data.iloc[:, 1:].apply(lambda row: row.notna().sum(), axis=1) <= median_t0))]
data

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663,target
6632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258650,0
3285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217,0
1824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7605000.0,7605000.0,NaN,NaN,NaN,7605000.0,7605000.0,7605000.0,0.256019,0
6135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8878285.0,8878285.0,8878285.0,8878285.0,8878285.0,8878285.0,8878285.0,8878285.0,0.264229,0
11625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5731213.8,5731213.8,5731213.8,5731213.8,5731213.8,5731213.8,5731213.8,5731213.8,0.256261,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217,0
6299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217,0
8101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6056700.0,6056700.0,NaN,NaN,NaN,6056700.0,6056700.0,6056700.0,0.256261,0
11317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3000000.0,3000000.0,NaN,NaN,NaN,3000000.0,3000000.0,3000000.0,0.256261,0


------------

In [17]:
target = data['target']
data = data.drop(['target'], axis=1)

-----

Проход окном

In [18]:
# Дропаем колонки в которых только NaN
good_data = data.drop(data.columns[data.isna().all()], axis=1)

In [19]:
# Серднее окном в 2 колонки
mean_gd = (good_data.count() + good_data.count().shift(-1) + good_data.count().shift(1))/3
mean_gd

col1               NaN
col2          6.000000
col3          9.000000
col4          7.666667
col5          4.666667
              ...     
col2659    1332.666667
col2660    2360.333333
col2661    3388.000000
col2662    3849.333333
col2663            NaN
Length: 2565, dtype: float64

In [20]:
selcol = good_data.columns[good_data.count()>mean_gd] ########### > || >= фичи х2 ~1300
selcol = selcol.to_list()
selcol.append(data.columns[-1]) # добавляем последнюю колонку

In [21]:
good_data = good_data[selcol]
good_data

,col3,col4,col8,col12,col16,col20,col24,col28,col32,col36,...,col2636,col2638,col2642,col2645,col2648,col2651,col2654,col2656,col2660,col2663
6632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258650
3285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217
1824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7605000.0,7605000.0,7605000.0,7605000.0,0.256019
6135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8878285.0,8878285.0,8878285.0,8878285.0,8878285.0,8878285.0,8878285.0,0.264229
11625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1166077.33,1166077.33,5731213.8,5731213.8,5731213.8,5731213.8,5731213.8,5731213.8,5731213.8,0.256261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217
6299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217
8101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6056700.0,6056700.0,6056700.0,0.256261
11317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1169869.72,1169869.72,NaN,NaN,NaN,3000000.0,3000000.0,3000000.0,3000000.0,0.256261


-------

In [22]:
# Колонки с категориальными данными
catCols = [col for col in good_data.columns if good_data[col].dtype=="object"]
catCols[-10:-1]
#print(f'number cat columns = {len(catCols)}')

['col1278',
 'col1280',
 'col1448',
 'col1649',
 'col1652',
 'col1654',
 'col2192',
 'col2195',
 'col2196']

In [23]:
categorical_columns = good_data[catCols]
numeric_columns = good_data.drop(catCols, axis=1)

-------

In [24]:
# Нормализация данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
transform_gd = scaler.fit_transform(numeric_columns)

In [25]:
Test_target = Test_data['target']
Test_data2 = Test_data[numeric_columns.columns]
Test_data3 = scaler.transform(Test_data2)

In [26]:
Test_data3.shape

(4337, 541)

In [27]:
# TT split
#X_train, X_test, y_train, y_test = train_test_split(transform_gd, target, test_size=0.2, random_state=rs+20)

In [28]:
# from sklearn.compose import make_column_selector, make_column_transformer
# from sklearn.pipeline import make_pipeline
# import numpy as np

# from sklearn.preprocessing import OneHotEncoder

# one_hot_encoder = make_column_transformer(
#     (
#         OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
#         make_column_selector(dtype_include="object"),
#     ),
#     remainder="passthrough",
# )

# hist_one_hot = make_pipeline(
#     one_hot_encoder, HistGradientBoostingClassifier(random_state=rs)
# )

In [29]:
transform_gd.shape

(4772, 541)

In [30]:
Test_data3.shape

(4337, 541)

In [31]:
gbc= HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=225, l2_regularization=0.15, max_leaf_nodes=60, random_state=rs+20)
gbc.fit(transform_gd,target)

roc_auc_score(Test_target, gbc.predict(Test_data3))


0.7177219537734559

In [32]:
f1_score(Test_target, gbc.predict(Test_data3))

0.17519685039370078

______________________________________________________________

try CatBoost


In [33]:
Test_data
Train_data

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663,target
10335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,0
6632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258650,0
3285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217,0
5674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,0
3440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,0
7744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,0
4068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,0
10150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7800000.0,7800000.0,NaN,NaN,NaN,7800000.0,7800000.0,7800000.0,0.269510,0


In [34]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=50,
    random_seed=4,
    learning_rate=0.5,
    custom_loss=['AUC']
)

clf.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    verbose=False,
    plot=True
)

NameError: name 'cat' is not defined

In [ ]:
from sklearn.model_selection import cross_validate
scoring = "f1"
one_hot_result = cross_validate(hist_one_hot, X_train, y_train, cv=3, scoring=scoring)

In [ ]:
one_hot_result['test_score']

-------

In [ ]:
columns_with_date = data.columns[data.isin(['самолет']).any()]
columns_with_date

In [ ]:
xxx = raw_data.loc[raw_data['client_id'] == 1969 ].sort_values(by='report_date')
xxx

In [ ]:
raw_data['col582'].value_counts()